## Install Pyspark

In [1]:
# !pip install pyspark

## Import required Library 

In [2]:
from pyspark.sql import SparkSession
import numpy as np 
import pandas as pd 
import warnings
from pyspark.sql import functions as F

## Configure Spark Session and Properties

In [3]:
%matplotlib inline
warnings.filterwarnings("ignore")

spark = SparkSession.builder \
          .master("local[*]") \
          .appName("Facts Analysis") \
          .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

23/01/12 13:09:16 WARN Utils: Your hostname, Abhisheks-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.3 instead (on interface en0)
23/01/12 13:09:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/12 13:09:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

## Prepare Data

### Creating Pandas Dataframe

In [5]:
#pandas_df_15=pd.read_csv("/kaggle/input/world-happiness/2015.csv")
#pandas_df_16=pd.read_csv("/kaggle/input/world-happiness/2016.csv")
#pandas_df_17=pd.read_csv("/kaggle/input/world-happiness/2017.csv")
#pandas_df_18=pd.read_csv("/kaggle/input/world-happiness/2018.csv")
#pandas_df_19=pd.read_csv("/kaggle/input/world-happiness/2019.csv")

In [6]:
pandas_df_15=pd.read_csv("/Users/abhi011097/Workspace/Dataset_Analysis/4_World_Happiness_Report/Dataset/2015.csv")
pandas_df_16=pd.read_csv("/Users/abhi011097/Workspace/Dataset_Analysis/4_World_Happiness_Report/Dataset/2016.csv")
pandas_df_17=pd.read_csv("/Users/abhi011097/Workspace/Dataset_Analysis/4_World_Happiness_Report/Dataset/2017.csv")
pandas_df_18=pd.read_csv("/Users/abhi011097/Workspace/Dataset_Analysis/4_World_Happiness_Report/Dataset/2018.csv")
pandas_df_19=pd.read_csv("/Users/abhi011097/Workspace/Dataset_Analysis/4_World_Happiness_Report/Dataset/2019.csv")

### Creating Spark Dataframe

In [7]:
spark_df_15=spark.createDataFrame(pandas_df_15)
spark_df_16=spark.createDataFrame(pandas_df_16)
spark_df_17=spark.createDataFrame(pandas_df_17)
spark_df_18=spark.createDataFrame(pandas_df_18)
spark_df_19=spark.createDataFrame(pandas_df_19)

### Creating Spark Table 

In [8]:
spark_df_15.registerTempTable("spark_tbl_15")
spark_df_16.registerTempTable("spark_tbl_16")
spark_df_17.registerTempTable("spark_tbl_17")
spark_df_18.registerTempTable("spark_tbl_18")
spark_df_19.registerTempTable("spark_tbl_19")

### Check Column Names

In [9]:
print("pandas_df_15 -> ",pandas_df_15.columns.to_list())
print("pandas_df_16 -> ",pandas_df_16.columns.to_list())
print("pandas_df_17 -> ",pandas_df_17.columns.to_list())
print("pandas_df_18 -> ",pandas_df_18.columns.to_list())
print("pandas_df_19 -> ",pandas_df_19.columns.to_list())

pandas_df_15 ->  ['Country', 'Region', 'Happiness Rank', 'Happiness Score', 'Standard Error', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual']
pandas_df_16 ->  ['Country', 'Region', 'Happiness Rank', 'Happiness Score', 'Lower Confidence Interval', 'Upper Confidence Interval', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual']
pandas_df_17 ->  ['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high', 'Whisker.low', 'Economy..GDP.per.Capita.', 'Family', 'Health..Life.Expectancy.', 'Freedom', 'Generosity', 'Trust..Government.Corruption.', 'Dystopia.Residual']
pandas_df_18 ->  ['Overall rank', 'Country or region', 'Score', 'GDP per capita', 'Social support', 'Healthy life expectancy', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption']
pandas_df_19 ->  ['Overall rank', 'Cou

## Validating Data load 

In [10]:
pandas_df_15.head(2)

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201


In [11]:
spark_df_15.show(2)

+-----------+--------------+--------------+---------------+--------------+------------------------+-------+------------------------+-------+-----------------------------+----------+-----------------+
|    Country|        Region|Happiness Rank|Happiness Score|Standard Error|Economy (GDP per Capita)| Family|Health (Life Expectancy)|Freedom|Trust (Government Corruption)|Generosity|Dystopia Residual|
+-----------+--------------+--------------+---------------+--------------+------------------------+-------+------------------------+-------+-----------------------------+----------+-----------------+
|Switzerland|Western Europe|             1|          7.587|       0.03411|                 1.39651|1.34951|                 0.94143|0.66557|                      0.41978|   0.29678|          2.51738|
|    Iceland|Western Europe|             2|          7.561|       0.04884|                 1.30232|1.40223|                 0.94784|0.62877|                      0.14145|    0.4363|          2.70201|


In [12]:
spark.sql("Select * from spark_tbl_15 limit 2").show(2)

+-----------+--------------+--------------+---------------+--------------+------------------------+-------+------------------------+-------+-----------------------------+----------+-----------------+
|    Country|        Region|Happiness Rank|Happiness Score|Standard Error|Economy (GDP per Capita)| Family|Health (Life Expectancy)|Freedom|Trust (Government Corruption)|Generosity|Dystopia Residual|
+-----------+--------------+--------------+---------------+--------------+------------------------+-------+------------------------+-------+-----------------------------+----------+-----------------+
|Switzerland|Western Europe|             1|          7.587|       0.03411|                 1.39651|1.34951|                 0.94143|0.66557|                      0.41978|   0.29678|          2.51738|
|    Iceland|Western Europe|             2|          7.561|       0.04884|                 1.30232|1.40223|                 0.94784|0.62877|                      0.14145|    0.4363|          2.70201|


## Facts

### Find the countries which never observed a drop in rank along with year wise rank and diffence of rank from 2015 to 2019 
* Sort the data by biggest to lowest change between 2015 to 2019

#### Using Spark Sql

In [13]:
spark.sql("""Select t1.Country ,t1.`Happiness Rank` as Rank_2015,t2.`Happiness Rank` Rank_2016,t3.`Happiness.Rank`  Rank_2017, 
            t4.`Overall rank` Rank_2018 ,t5.`Overall rank` Rank_2019,
            (t1.`Happiness Rank`-t5.`Overall rank`) Increment_Factor
            from  spark_tbl_15  t1
            inner join spark_tbl_16 t2
            on t1.Country = t2.Country and t1.`Happiness Rank` >= t2.`Happiness Rank`
            inner join spark_tbl_17 t3
            on t1.Country = t3.Country and t2.`Happiness Rank` >= t3.`Happiness.Rank`
            inner join spark_tbl_18 t4
            on t1.Country = t4.`Country or region` and t3.`Happiness.Rank` >= t4.`Overall rank`
            inner join spark_tbl_19 t5
            on t1.Country = t5.`Country or region` and t4.`Overall rank` >= t5.`Overall rank`
            order by Increment_Factor desc
          
          """).show(200)

+-------------------+---------+---------+---------+---------+---------+----------------+
|            Country|Rank_2015|Rank_2016|Rank_2017|Rank_2018|Rank_2019|Increment_Factor|
+-------------------+---------+---------+---------+---------+---------+----------------+
|              Benin|      155|      153|      143|      136|      102|              53|
|        Ivory Coast|      151|      139|      128|      107|       99|              52|
|           Honduras|      105|      104|       91|       72|       59|              46|
|            Hungary|      104|       91|       75|       69|       62|              42|
|            Romania|       86|       71|       57|       52|       48|              38|
|       Burkina Faso|      152|      145|      134|      121|      115|              37|
|           Cameroon|      133|      114|      107|       99|       96|              37|
|           Bulgaria|      134|      129|      105|      100|       97|              37|
|Congo (Brazzaville)|

#### Using Pandas

In [14]:
temp_res=pd.merge(pandas_df_15[['Country','Happiness Rank']], pandas_df_16[['Country','Happiness Rank']], on="Country")\
        .rename(columns={'Happiness Rank_x':'Rank_2015','Happiness Rank_y':'Rank_2016'})\
        .query("Rank_2015 >= Rank_2016")\
        .merge(pandas_df_17[['Country','Happiness.Rank']],on="Country")\
        .rename(columns={'Happiness.Rank':'Rank_2017'})\
        .query("Rank_2016 >= Rank_2017")\
        .merge(pandas_df_18[['Country or region','Overall rank']],left_on="Country",right_on="Country or region")\
        .rename(columns={'Overall rank':'Rank_2018'})\
        .query("Rank_2017 >= Rank_2018")\
        .drop(columns='Country or region')\
        .merge(pandas_df_19[['Country or region','Overall rank']],left_on="Country",right_on="Country or region")\
        .rename(columns={'Overall rank':'Rank_2019'})\
        .query("Rank_2018 >= Rank_2019")\
        .drop(columns='Country or region')
        

temp_res.assign(Increment_Factor=temp_res["Rank_2015"] - temp_res["Rank_2019"])\
        .sort_values('Increment_Factor',ascending=False)\
        .style.hide_index()

Country,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Increment_Factor
Benin,155,153,143,136,102,53
Ivory Coast,151,139,128,107,99,52
Honduras,105,104,91,72,59,46
Hungary,104,91,75,69,62,42
Romania,86,71,57,52,48,38
Cameroon,133,114,107,99,96,37
Burkina Faso,152,145,134,121,115,37
Bulgaria,134,129,105,100,97,37
Cambodia,145,140,129,120,109,36
Congo (Brazzaville),139,127,124,114,103,36


#### Using Spark Dataframe

In [15]:
spark_df_15['Country','Happiness Rank']\
    .withColumnRenamed("Happiness Rank","Rank_2015")\
    .join(spark_df_16['Country','Happiness Rank'].withColumnRenamed("Happiness Rank","Rank_2016"),on='Country')\
    .filter(F.col("Rank_2015") >= F.col("Rank_2016"))\
    .join(spark_df_17['Country','`Happiness.Rank`'].withColumnRenamed("Happiness.Rank","Rank_2017"),on='Country')\
    .filter(F.col("Rank_2016") >= F.col("Rank_2017"))\
    .join(spark_df_18['`Country or region`','`Overall rank`'].withColumnRenamed("Country or region","Country").withColumnRenamed("Overall rank","Rank_2018"),on='Country')\
    .filter(F.col("Rank_2017") >= F.col("Rank_2018"))\
    .join(spark_df_19['`Country or region`','`Overall rank`'].withColumnRenamed("Country or region","Country").withColumnRenamed("Overall rank","Rank_2019"),on='Country')\
    .filter(F.col("Rank_2018") >= F.col("Rank_2019"))\
    .withColumn('Increment_Factor', F.col("Rank_2015") - F.col("Rank_2019"))\
    .sort('Increment_Factor',ascending=False)\
    .show(200)
    

+-------------------+---------+---------+---------+---------+---------+----------------+
|            Country|Rank_2015|Rank_2016|Rank_2017|Rank_2018|Rank_2019|Increment_Factor|
+-------------------+---------+---------+---------+---------+---------+----------------+
|              Benin|      155|      153|      143|      136|      102|              53|
|        Ivory Coast|      151|      139|      128|      107|       99|              52|
|           Honduras|      105|      104|       91|       72|       59|              46|
|            Hungary|      104|       91|       75|       69|       62|              42|
|            Romania|       86|       71|       57|       52|       48|              38|
|       Burkina Faso|      152|      145|      134|      121|      115|              37|
|           Cameroon|      133|      114|      107|       99|       96|              37|
|           Bulgaria|      134|      129|      105|      100|       97|              37|
|           Cambodia|

### Find the countries which continously observed drop in rank along with year wise rank and diffence of rank from 2015 to 2019 
* Sort the data by biggest to lowest change between 2015 to 2019

#### Using Spark Sql

In [16]:
spark.sql("""Select t1.Country ,t1.`Happiness Rank` as Rank_2015,t2.`Happiness Rank` Rank_2016,t3.`Happiness.Rank`  Rank_2017, 
            t4.`Overall rank` Rank_2018 ,t5.`Overall rank` Rank_2019,
            (t1.`Happiness Rank`-t5.`Overall rank`) Decrement_Factor
            from  spark_tbl_15  t1
            inner join spark_tbl_16 t2
            on t1.Country = t2.Country and t1.`Happiness Rank` < t2.`Happiness Rank`
            inner join spark_tbl_17 t3
            on t1.Country = t3.Country and t2.`Happiness Rank` < t3.`Happiness.Rank`
            inner join spark_tbl_18 t4
            on t1.Country = t4.`Country or region` and t3.`Happiness.Rank` < t4.`Overall rank`
            inner join spark_tbl_19 t5
            on t1.Country = t5.`Country or region` and t4.`Overall rank` < t5.`Overall rank`
            order by Decrement_Factor asc
          
          """).show(200)

+-----------+---------+---------+---------+---------+---------+----------------+
|    Country|Rank_2015|Rank_2016|Rank_2017|Rank_2018|Rank_2019|Decrement_Factor|
+-----------+---------+---------+---------+---------+---------+----------------+
|  Venezuela|       23|       44|       82|      102|      108|             -85|
|     Zambia|       85|      106|      116|      125|      138|             -53|
|   Zimbabwe|      115|      131|      138|      144|      146|             -31|
|      India|      117|      118|      122|      133|      140|             -23|
|    Belarus|       59|       61|       67|       73|       81|             -22|
|   Botswana|      128|      137|      142|      146|      148|             -20|
|     Malawi|      131|      132|      136|      147|      150|             -19|
|    Moldova|       52|       55|       56|       67|       71|             -19|
|     Brazil|       16|       17|       22|       28|       32|             -16|
| Azerbaijan|       80|     

#### Using Pandas

In [17]:

temp_res=pd.merge(pandas_df_15[['Country','Happiness Rank']], pandas_df_16[['Country','Happiness Rank']], on="Country")\
        .rename(columns={'Happiness Rank_x':'Rank_2015','Happiness Rank_y':'Rank_2016'})\
        .query("Rank_2015 < Rank_2016")\
        .merge(pandas_df_17[['Country','Happiness.Rank']],on="Country")\
        .rename(columns={'Happiness.Rank':'Rank_2017'})\
        .query("Rank_2016 < Rank_2017")\
        .merge(pandas_df_18[['Country or region','Overall rank']],left_on="Country",right_on="Country or region")\
        .rename(columns={'Overall rank':'Rank_2018'})\
        .query("Rank_2017 < Rank_2018")\
        .drop(columns='Country or region')\
        .merge(pandas_df_19[['Country or region','Overall rank']],left_on="Country",right_on="Country or region")\
        .rename(columns={'Overall rank':'Rank_2019'})\
        .query("Rank_2018 < Rank_2019")\
        .drop(columns='Country or region')
        

temp_res.assign(Decrement_Factor=temp_res["Rank_2015"] - temp_res["Rank_2019"])\
        .sort_values('Decrement_Factor')\
        .style.hide_index()

Country,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Decrement_Factor
Venezuela,23,44,82,102,108,-85
Zambia,85,106,116,125,138,-53
Zimbabwe,115,131,138,144,146,-31
India,117,118,122,133,140,-23
Belarus,59,61,67,73,81,-22
Botswana,128,137,142,146,148,-20
Moldova,52,55,56,67,71,-19
Malawi,131,132,136,147,150,-19
Brazil,16,17,22,28,32,-16
Azerbaijan,80,81,85,87,90,-10


#### Using Spark Dataframe

In [18]:
spark_df_15['Country','Happiness Rank']\
    .withColumnRenamed("Happiness Rank","Rank_2015")\
    .join(spark_df_16['Country','Happiness Rank'].withColumnRenamed("Happiness Rank","Rank_2016"),on='Country')\
    .filter(F.col("Rank_2015") < F.col("Rank_2016"))\
    .join(spark_df_17['Country','`Happiness.Rank`'].withColumnRenamed("Happiness.Rank","Rank_2017"),on='Country')\
    .filter(F.col("Rank_2016") < F.col("Rank_2017"))\
    .join(spark_df_18['`Country or region`','`Overall rank`'].withColumnRenamed("Country or region","Country").withColumnRenamed("Overall rank","Rank_2018"),on='Country')\
    .filter(F.col("Rank_2017") < F.col("Rank_2018"))\
    .join(spark_df_19['`Country or region`','`Overall rank`'].withColumnRenamed("Country or region","Country").withColumnRenamed("Overall rank","Rank_2019"),on='Country')\
    .filter(F.col("Rank_2018") < F.col("Rank_2019"))\
    .withColumn('Decrement_Factor', F.col("Rank_2015") - F.col("Rank_2019"))\
    .sort('Decrement_Factor')\
    .show(200)
    

+-----------+---------+---------+---------+---------+---------+----------------+
|    Country|Rank_2015|Rank_2016|Rank_2017|Rank_2018|Rank_2019|Decrement_Factor|
+-----------+---------+---------+---------+---------+---------+----------------+
|  Venezuela|       23|       44|       82|      102|      108|             -85|
|     Zambia|       85|      106|      116|      125|      138|             -53|
|   Zimbabwe|      115|      131|      138|      144|      146|             -31|
|      India|      117|      118|      122|      133|      140|             -23|
|    Belarus|       59|       61|       67|       73|       81|             -22|
|   Botswana|      128|      137|      142|      146|      148|             -20|
|     Malawi|      131|      132|      136|      147|      150|             -19|
|    Moldova|       52|       55|       56|       67|       71|             -19|
|     Brazil|       16|       17|       22|       28|       32|             -16|
| Azerbaijan|       80|     

### In progress

In [19]:
spark.stop()